In [13]:
import os
import pandas as pd
import gspread
import sqlalchemy
from sqlalchemy import text
from google.oauth2.service_account import Credentials
from google.cloud.sql.connector import Connector, IPTypes


# ==============================
# CONFIGURATION
# ==============================

SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SERVICE_ACCOUNT_FILE = "/Users/deepshah/Downloads/tiffinstash-key.json"

INSTANCE_CONNECTION_NAME = "pelagic-campus-484800-b3:us-central1:tiffinstash-master"
DB_USER = "postgres"
DB_PASSWORD = "tiffinstash2026"
DB_NAME = "postgres"


# ==============================
# FUNCTION 1: READ GOOGLE SHEET
# ==============================

def fetch_sheet_as_df(sheet_id: str, sheet_name: str) -> pd.DataFrame:
    """
    Reads a specific worksheet (by sheet_name) from a Google Sheet
    and returns it as a Pandas DataFrame.
    """
    try:
        creds = Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE,
            scopes=SCOPES
        )
        client = gspread.authorize(creds)

        spreadsheet = client.open_by_key(sheet_id)
        worksheet = spreadsheet.worksheet(sheet_name)

        data = worksheet.get_all_records()
        df = pd.DataFrame(data)

        print(f"✅ Fetched {len(df)} rows from '{sheet_name}'")
        return df

    except Exception as e:
        print(f"❌ Error reading sheet '{sheet_name}': {e}")
        return pd.DataFrame()


# ==================================
# FUNCTION 2: EXPORT TO POSTGRES
# ==================================

def export_df_to_postgres(df: pd.DataFrame, table_name: str):
    """
    Exports a DataFrame to Google Cloud SQL (PostgreSQL).
    Replaces the table if it already exists.
    """
    if df.empty:
        print("⚠️ DataFrame is empty. Nothing to export.")
        return

    # Attempt datetime conversion
    for col in df.columns:
        if df[col].dtype == "object":
            try:
                df[col] = pd.to_datetime(df[col], dayfirst=True, format="mixed")
            except (ValueError, TypeError):
                continue

    credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
    connector = Connector(credentials=credentials)

    def getconn():
        return connector.connect(
            INSTANCE_CONNECTION_NAME,
            "pg8000",
            user=DB_USER,
            password=DB_PASSWORD,
            db=DB_NAME,
            ip_type=IPTypes.PUBLIC
        )

    engine = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn
    )

    try:
        df.to_sql(table_name, engine, if_exists="replace", index=False)
        print(f"✅ Imported {len(df)} rows into '{table_name}'")

        with engine.connect() as conn:
            result = conn.execute(text(f'SELECT COUNT(*) FROM "{table_name}"'))
            print(f"🔎 Verified row count: {result.fetchone()[0]}")

    except Exception as e:
        print(f"❌ Import failed: {e}")

    finally:
        connector.close()


In [14]:

# ==============================
# SELLER EXTENSIONS
# ==============================

if __name__ == "__main__":

    SELLER_EXTENSIONS_SHEET_ID = "1Awva-WPyvhT0XHYm53YdfmyZW-xLYM8HlWCb77kET5Q"

    seller_extensions_df = fetch_sheet_as_df(
        SELLER_EXTENSIONS_SHEET_ID,
        sheet_name="Sheet1"
    )

    export_df_to_postgres(seller_extensions_df, "seller-extensions")


✅ Fetched 29 rows from 'Sheet1'
✅ Imported 29 rows into 'seller-extensions'
🔎 Verified row count: 29


In [16]:

# ==============================
# SKU Sheet
# ==============================

if __name__ == "__main__":

    SELLER_EXTENSIONS_SHEET_ID = "1Awva-WPyvhT0XHYm53YdfmyZW-xLYM8HlWCb77kET5Q"

    sku_df = fetch_sheet_as_df(
        SELLER_EXTENSIONS_SHEET_ID,
        sheet_name="Sheet2"
    )

    export_df_to_postgres(sku_df, "sku-info")


✅ Fetched 3011 rows from 'Sheet2'
✅ Imported 3011 rows into 'sku-info'
🔎 Verified row count: 3011
